# Hautkrebs-Klassifikationsmodell mit EfficientNet und MLP

## Einführung

Dieses Modell kombiniert ein auf EfficientNet basierendes CNN (Convolutional Neural Network) mit einem MLP (Multilayer Perceptron), um sowohl Bilder von Hautläsionen als auch tabellarische Daten (Alter und Geschlecht) zu verarbeiten. Das Ziel ist es, die Genauigkeit bei der Klassifizierung von gutartigen und bösartigen Läsionen zu verbessern.

## Implementierung

### Import der Bibliotheken
```python
import torch
import torch.nn as nn
import torchvision.models as models
```

### Definition des Modells
```python
class SkinCancerModel(nn.Module):
    def __init__(self, num_classes=2):
        super(SkinCancerModel, self).__init__()
        
        # **CNN für Bilder (EfficientNet)**
        self.cnn = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
        self.cnn.classifier = nn.Identity()  # Entfernt die letzte Schicht

        # **MLP für tabellarische Daten (Alter und Geschlecht)**
        self.mlp = nn.Sequential(
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU()
        )

        # **Automatische Berechnung der Ausgabegröße von EfficientNet**
        dummy_input = torch.randn(1, 3, 224, 224)
        img_output_size = self.cnn(dummy_input).shape[-1]  # Berechnet die korrekte Größe

        # **Finale kombinierte Klassifikationsschicht**
        self.classifier = nn.Sequential(
            nn.Linear(img_output_size + 8, 256),  # Dynamisch angepasst
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, image, metadata):
        img_features = self.cnn(image)  
        meta_features = self.mlp(metadata)
        combined = torch.cat((img_features, meta_features), dim=1)  
        output = self.classifier(combined)
        return output
```

## Erklärung des Modells

1. **EfficientNet-B0**:
   - Wird als Merkmalsextraktor für Bilder von Hautläsionen verwendet.
   - Die letzte Schicht (`classifier`) wird entfernt, um einen Feature-Vektor zu erhalten.

2. **MLP für tabellarische Daten**:
   - Wird verwendet, um zusätzliche Variablen wie Alter und Geschlecht zu verarbeiten.
   - Enthält zwei vollständig verbundene Schichten mit ReLU-Aktivierungen.

3. **Kombination der Merkmale**:
   - Die Merkmale der Bilder und der tabellarischen Daten werden zusammengeführt.
   - Sie werden durch eine dichte Schicht mit 256 Neuronen geleitet, gefolgt von einer Ausgabeschicht für die Klassifikation.

## Verwendung des Modells
```python
# Eine Instanz des Modells erstellen
model = SkinCancerModel(num_classes=2)

# Beispielhafte Eingabedaten
image_input = torch.randn(1, 3, 224, 224)  # Eingabebild
metadata_input = torch.randn(1, 2)  # Alter und Geschlecht

# Forward pass
output = model(image_input, metadata_input)
print(output)
```

## Fazit
Dieses Modell ermöglicht eine robustere Klassifizierung von Hautkrebs, indem es zusätzliche Daten zu den Bildern einbezieht. Die Kombination von EfficientNet mit einem MLP verbessert die Vorhersagefähigkeit, indem relevante zusätzliche Informationen berücksichtigt werden.

---

**Hinweis**: Diese Dokumentation ist als lebendes Dokument gedacht. Wenn sich das Modell weiterentwickelt, werden Aktualisierungen vorgenommen, um neue Funktionen, Verbesserungen und Änderungen in den Best Practices widerzuspiegeln.